In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle
import argparse

parser = argparse.ArgumentParser(description='This is a demonstration program')

# Here we add an argument to the parser, specifying the expected type, a help message, etc.
# parser.add_argument('-batch_size', type=str, required=True, help='Please provide a batch_size')

# args = parser.parse_args()

# Now we can use the argument value in our program.
# print(f'batch size: {args.batch_size}')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# batch_size = args.batch_size # to use the batch_size cmd arg -> python file_name.py -batch_size 32
batch_size = 64
block_size = 128
max_iters = 3000
learning_rate = 3e-4
eval_iters = 100
n_embd = 384
n_head = 8
n_layer = 8
dropout = 0.2

print(device)

cuda


In [2]:
chars = ""
with open('wizard_of_oz.txt' , 'r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(set(text))
#print(text[:200])

vocab_size= len(chars)
vocab_size

89

In [3]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

#encoded_hello = encode('hello')
#decoded_hello = decode(encoded_hello)
#decoded_hello

data = torch.tensor(encode(text), dtype=torch.long)
print (data[:100])

tensor([88, 46, 63, 74, 66, 59, 24,  1, 46, 62, 59,  1, 49, 69, 68, 58, 59, 72,
        60, 75, 66,  1, 49, 63, 80, 55, 72, 58,  1, 69, 60,  1, 41, 80,  0,  0,
         0, 27, 75, 74, 62, 69, 72, 24,  1, 38, 12,  1, 32, 72, 55, 68, 65,  1,
        28, 55, 75, 67,  0,  0, 44, 59, 66, 59, 55, 73, 59,  1, 58, 55, 74, 59,
        24,  1, 32, 59, 56, 72, 75, 55, 72, 79,  1, 15, 10,  1, 15, 23, 23, 17,
         1, 53, 59, 28, 69, 69, 65,  1,  3, 19])


In [4]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[61, 69, 66,  ..., 73, 55, 63],
        [72, 59, 55,  ..., 59, 58,  1],
        [59, 59, 74,  ..., 73, 75, 68],
        ...,
        [66, 63, 59,  ..., 55, 63, 58],
        [73, 59, 58,  ..., 73, 74, 63],
        [12,  1, 45,  ..., 59,  1, 68]], device='cuda:0')
targets:
tensor([[69, 66, 58,  ..., 55, 63, 58],
        [59, 55, 74,  ..., 58,  1, 67],
        [59, 74,  0,  ..., 75, 68,  1],
        ...,
        [63, 59, 58,  ..., 63, 58,  1],
        [59, 58,  1,  ..., 74, 63, 66],
        [ 1, 45, 69,  ...,  1, 68, 63]], device='cuda:0')


In [5]:
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([88]) target is tensor(46)
when input is tensor([88, 46]) target is tensor(63)
when input is tensor([88, 46, 63]) target is tensor(74)
when input is tensor([88, 46, 63, 74]) target is tensor(66)
when input is tensor([88, 46, 63, 74, 66]) target is tensor(59)
when input is tensor([88, 46, 63, 74, 66, 59]) target is tensor(24)
when input is tensor([88, 46, 63, 74, 66, 59, 24]) target is tensor(1)
when input is tensor([88, 46, 63, 74, 66, 59, 24,  1]) target is tensor(46)


In [6]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [7]:
class Head(nn.Module):
    """ one head of self-attention """
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out


# [1, 0, 0]
# [1, 0.6, 0]
# [1, 0.6, 0.4]
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3])
        out = self.dropout(self.proj(out))
        return out

    
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """  
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)
    
    
class Block(nn.Module):
    """ Transformer block: communication followed by computation """
    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x



class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head = n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) #final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.0)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.2)
            
    def forward(self, index, targets=None):
        #logits = self.token_embedding_table(index)        
        B, T  = index.shape
        #idx and targets are both (B,T) tensors of integers
        tok_embed = self.token_embedding_table(index) # (B, T, C)
        pos_embed = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_embed + pos_embed # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)
                                      
        if targets is None:
            loss = None
        else:        
            #batch, time, channel
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index


model = GPTLanguageModel(vocab_size)
m = model.to(device)

#context = torch.zeros((1,1), dtype=torch.long, device=device)
#generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
#print(generated_chars)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.489, val loss: 4.489
step: 100, train loss: 2.614, val loss: 3.017
step: 200, train loss: 2.441, val loss: 2.950
step: 300, train loss: 2.413, val loss: 2.953
step: 400, train loss: 2.393, val loss: 2.985
step: 500, train loss: 2.376, val loss: 2.986
step: 600, train loss: 2.385, val loss: 2.967
step: 700, train loss: 2.368, val loss: 3.006
step: 800, train loss: 2.363, val loss: 3.010
step: 900, train loss: 2.364, val loss: 3.019
step: 1000, train loss: 2.362, val loss: 3.019
step: 1100, train loss: 2.360, val loss: 3.037
step: 1200, train loss: 2.362, val loss: 3.036
step: 1300, train loss: 2.359, val loss: 3.043
step: 1400, train loss: 2.358, val loss: 3.020
step: 1500, train loss: 2.355, val loss: 3.021


In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)